In [116]:
import tkinter as tk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [117]:
file = pd.read_csv("admission_data.csv")
file.rename(columns={'Chance of Admit ': 'Chance of Admit'}, inplace=True)


In [118]:
file[file["Chance of Admit"] >= 0.9]


,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
5,330,115,5,4.5,3.0,9.34,1,0.90
22,328,116,5,5.0,5.0,9.50,1,0.94
23,334,119,5,5.0,4.5,9.70,1,0.95
24,336,119,5,4.0,3.5,9.80,1,0.97
...,...,...,...,...,...,...,...,...
446,327,118,4,5.0,5.0,9.67,1,0.93
452,328,116,4,5.0,3.5,9.60,1,0.93
472,327,116,4,4.0,4.5,9.48,1,0.90
496,337,117,5,5.0,5.0,9.87,1,0.96


PREDICT CHANCE OF ADMISSION IN UNIVERSITY

In [119]:
Chance_of_admit = pd.DataFrame(file["Chance of Admit"])
GRE_Score = pd.DataFrame(file["GRE Score"])
mod_file = file.drop(["Chance of Admit"], axis="columns")
mod_file


,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332,108,5,4.5,4.0,9.02,1
496,337,117,5,5.0,5.0,9.87,1
497,330,120,5,4.5,5.0,9.56,1
498,312,103,4,4.0,5.0,8.43,0


In [120]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
kf = KFold(n_splits=200)

scores_linear = [0]
scores_logistic = [0]
scores_svm = [0]
scores_rf = [0]


def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)


f_X_train, f_X_test, f_y_train, f_y_test = 0, 0, 0, 0
for train_index, test_index in kf.split(mod_file):
    X_train, X_test, y_train, y_test = mod_file.loc[train_index], mod_file.loc[
        test_index], Chance_of_admit.loc[train_index], Chance_of_admit.loc[test_index]
    res = round(get_score(LinearRegression(), X_train,
                          X_test, y_train, y_test)*100, 2)
    if res > max(scores_linear):
        f_X_train, f_X_test, f_y_train, f_y_test = X_train, X_test, y_train, y_test

    scores_linear.append(res)
f_X_train, f_X_test, f_y_train, f_y_test
model = LinearRegression()
model.fit(f_X_train, f_y_train)
print(round(model.score(f_X_test, f_y_test)*100, 2))
predicted = []
for a in np.array(model.predict(f_X_test)).flatten():
    predicted.append(round(a*100, 2))
print(f_X_test)
print(predicted)


99.67
     GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research
336        319          110                  3  3.0   2.5  8.79         0
337        332          118                  5  5.0   5.0  9.47         1
[72.81, 93.64]


GRAPH FOR CHANCE OF ADMISSION IN UNIVERSITY

In [121]:
# # d= {"GRE":319,"TOEFL":110,"University":3, "SOP":3.0,"LOR":2.5,"CGPA":8.79,"Research":0}
# # # d= {"GRE":332,"TOEFL":118,"University":5, "SOP":5.0,"LOR":5.0,"CGPA":9.47,"Research":1}
# df = pd.DataFrame({
#     'X': ["GRE","TOEFL","SOP","LOR","CGPA"],
#     'Y': [332,118,5.0,5.0,9.47],
#     'Z': [319,110,3.0,2.5,8.79]
# })

# ax = plt.subplots()

# # plotting columns
# ax = plt.bar(df["X"], df["Y"], width = 0.4)
# ax = plt.bar(df["X"]+0.4, df["Z"], width = 0.4)

# # renaming the axes
# plt.xlabel("")
# # visualizing illustration
# plt.show()


PREDICT HOW MUCH SCORES NEEDED TO GET INTO SPECIFIED RATING UNIVERSITY

In [122]:
file = pd.read_csv("admission_data.csv")
file.rename(columns={'Chance of Admit ': 'Chance of Admit'}, inplace=True)
U_rating = file["University Rating"]
mod_file_1 = file.drop(
    ["Chance of Admit", "University Rating"], axis="columns")
mod_file_1


,GRE Score,TOEFL Score,SOP,LOR,CGPA,Research
0,337,118,4.5,4.5,9.65,1
1,324,107,4.0,4.5,8.87,1
2,316,104,3.0,3.5,8.00,1
3,322,110,3.5,2.5,8.67,1
4,314,103,2.0,3.0,8.21,0
...,...,...,...,...,...,...
495,332,108,4.5,4.0,9.02,1
496,337,117,5.0,5.0,9.87,1
497,330,120,4.5,5.0,9.56,1
498,312,103,4.0,5.0,8.43,0


In [125]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
kf = KFold(n_splits=250)

scores_linear = [0]
scores_logistic = [0]
scores_svm = [0]
scores_rf = [0]


def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train.reshape(-1, 1), y_train)
    return model.score(X_test.reshape(-1, 1), y_test)


f_X_train, f_X_test, f_y_train, f_y_test = 0, 0, 0, 0
for train_index, test_index in kf.split(U_rating):
    X_train, X_test, y_train, y_test = np.array(U_rating.loc[train_index]), np.array(
        U_rating.loc[test_index]), np.array(mod_file_1.loc[train_index]), np.array(mod_file_1.loc[test_index])
    res = round(get_score(LinearRegression(), X_train,
                          X_test, y_train, y_test)*100, 2)
    if res > max(scores_linear):
        f_X_train, f_X_test, f_y_train, f_y_test = X_train, X_test, y_train, y_test
        scores_linear.append(res)

f_X_train, f_X_test, f_y_train, f_y_test = pd.DataFrame(f_X_train), pd.DataFrame(
    f_X_test), pd.DataFrame(f_y_train), pd.DataFrame(f_y_test)
model = LinearRegression()
model.fit(f_X_train, f_y_train)
print(round(model.score(f_X_test, f_y_test)*100, 2))
predicted, actual = [], []
print(np.array(model.predict(f_X_test)).flatten())
print(np.array(f_y_test).flatten())
for a in np.array(model.predict(f_X_test)).flatten():
    predicted.append(round(a, 0))

for a in np.array(f_y_test).flatten():
    actual.append(round(a, 0))
print(predicted)
print(actual)


86.5
[328.23145128 113.69993427   4.5641997    4.40868562   9.28056666
   0.90885924 309.50614467 103.34493235   2.67173762   2.93573209
   8.16005965   0.35477871]
[338.   115.     4.5    5.     9.23   1.   306.   103.     2.5    3.
   8.36   0.  ]
[328.0, 114.0, 5.0, 4.0, 9.0, 1.0, 310.0, 103.0, 3.0, 3.0, 8.0, 0.0]
[338.0, 115.0, 4.0, 5.0, 9.0, 1.0, 306.0, 103.0, 2.0, 3.0, 8.0, 0.0]


DISPLAY IMAGES

In [1]:
from PIL import ImageTk, Image
import tkinter as tk
def image(img):
    path = "K:\TRAINING PROJECT\PHOTOS"+"\\"
    im = path+str(img)
    return ImageTk.PhotoImage(Image.open(im))
window = tk.Tk()
window.title("University Graduates admission prediction system")
window.geometry("1028x720")
class temp:
    def __init__(self, frame):
        sub = tk.Frame(window)
        sub.pack()
        self.origin = tk.Label(sub, text="Made with",
                                compound='right', font=("Garamond", 16))
        self.origin.pack()
a = temp(window)
heart = image("Heart.png")
a.origin.configure(image=heart)
window.mainloop()
